In [1]:
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import pytz

In [2]:
season = 20232024
version = "300823_GW3"

In [3]:
#Import packages
import psycopg2 # python -> psql connection
import psycopg2.extras
from sqlalchemy import create_engine, text

#Import secret settings
import CJDH_local_settings
dbname = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['dbname']
user = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['user']
password = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['password']
host = "localhost"
port = 5432

#Create connection to psql via psycopg2 & sqlalchemy
connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(connection_string)
engine.autocommit = True  # read documentation understanding when to Use & NOT use # changes will be automatically committed to the database after each statement is executed.

## Player Summary DataFrame

In [4]:
#Create DF from FPL API Request
headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
get=requests.get("https://fantasy.premierleague.com/api/bootstrap-static/", headers=headers, timeout=5)
data=json.loads(get.text)

#player_raw = Summary stats for each player for the 2023/24 season
cols=list(data['elements'][0].keys())
player_raw = pd.DataFrame(data['elements'], columns = cols)

#Add a column for date that data is requested
#When we add new data, duplicates can be deleted with the most recent data preferred
player_raw['now_datetime_utc'] = datetime.now(tz=pytz.utc)

#Add current and next gw to player data
def get_gw(df, df_col:str):
    #Function to get the current gw from the events data
    #Input: Events DataFrame
    #Output: gameweek (INT)
    gw_id = df[df[df_col]==True]['id']
    if gw_id.empty==True:
        return int(0)
    else: 
        return int(gw_id.values)

eventdf = pd.DataFrame(data['events'])
player_raw['current_gw'] = get_gw(eventdf,df_col='is_current')
player_raw['next_gw'] = get_gw(eventdf,df_col='is_next')
player_raw['season'] = season

#Codes used to connect FPL & FBREF data which identify players with different unique codes
codes = pd.read_csv('data/2023_IDs.csv')[['code','22_FPL_code','fbref_id']]
player_raw = player_raw.merge(codes, how='left',left_on='code',right_on='code')

#Refine columns to be used in PSQL table
print(list(player_raw.columns))
player = player_raw[['fbref_id','code','season','id','web_name','team','element_type','status','now_cost','transfers_in','transfers_in_event','transfers_out','transfers_out_event','event_points','goals_scored', 'assists', 'clean_sheets', 'goals_conceded','total_points','minutes','points_per_game',
'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts','form','ep_this','ep_next', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded',
'influence_rank','creativity_rank','threat_rank','ict_index_rank','corners_and_indirect_freekicks_order','direct_freekicks_order','penalties_order',
'expected_goals_per_90', 'saves_per_90', 'expected_assists_per_90', 'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90', 'goals_conceded_per_90', 'points_per_game_rank', 'selected_rank', 'starts_per_90', 'clean_sheets_per_90','now_datetime_utc','current_gw','next_gw']]

['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded', 'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'thre

In [5]:
#Create staging table for player data 
def create_staging_table_player(cursor):
    cursor.execute(text("""
        DROP TABLE IF EXISTS player CASCADE;
        CREATE UNLOGGED TABLE player (
            fbref_id                TEXT,
            code                    NUMERIC, --static
            season                  NUMERIC,
            id                      NUMERIC, --season code
            web_name                TEXT,
            team                    TEXT,
            element_type            NUMERIC,
            status                  TEXT,
            now_cost                NUMERIC,
            transfers_in            NUMERIC,
            transfers_in_event      NUMERIC,
            transfers_out           NUMERIC,
            transfers_out_event     NUMERIC,
            event_points            NUMERIC,
            goals_scored            NUMERIC,
            assists                 NUMERIC,
            clean_sheets            NUMERIC,
            goals_conceded          NUMERIC,
            total_points            NUMERIC,
            minutes                 NUMERIC,
            points_per_game         NUMERIC,
            own_goals               NUMERIC,
            penalties_saved         NUMERIC,
            penalties_missed        NUMERIC,
            yellow_cards            NUMERIC,
            red_cards               NUMERIC,
            saves                   NUMERIC,
            bonus                   NUMERIC,
            bps                     NUMERIC,
            influence               NUMERIC,
            creativity              NUMERIC,
            threat                  NUMERIC,
            ict_index               NUMERIC,
            starts                  NUMERIC,
            form                    NUMERIC,
            ep_this                 NUMERIC,
            ep_next                 NUMERIC,
            expected_goals          NUMERIC,
            expected_assists        NUMERIC,
            expected_goal_involvements NUMERIC,
            expected_goals_conceded NUMERIC,
            influence_rank          NUMERIC,
            creativity_rank         NUMERIC,
            threat_rank             NUMERIC,
            ict_index_rank          NUMERIC,
            corners_and_indirect_freekicks_order NUMERIC,
            direct_freekicks_order  NUMERIC,
            penalties_order         NUMERIC,
            expected_goals_per_90   NUMERIC,
            saves_per_90            NUMERIC,
            expected_assists_per_90 NUMERIC,
            expected_goal_involvements_per_90 NUMERIC,
            expected_goals_conceded_per_90 NUMERIC,
            goals_conceded_per_90   NUMERIC,
            points_per_game_rank    NUMERIC,
            selected_rank           NUMERIC,
            starts_per_90           NUMERIC,
            clean_sheets_per_90     NUMERIC,
            now_datetime_UTC        TIMESTAMP,
            current_gw              NUMERIC,
            next_gw                 NUMERIC
        );"""))
# creating our schema  and sending the table to psql
with engine.connect() as cursor:
    create_staging_table_player(cursor)
player.to_sql('player', con=engine, if_exists='replace',index=False)

152

## Teams Data

In [6]:
#Adding team names to playersummarydf (according to the team #)
teams_raw = pd.DataFrame(data['teams'])
teams_raw['season'] = season
print(teams_raw.columns)
teams = teams_raw[['code','season','id','name','strength','strength_overall_home', 'strength_overall_away',
       'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
       'strength_defence_away']]

def create_staging_table_teams(cursor):
    cursor.execute(text("""
        DROP TABLE IF EXISTS teams CASCADE;
        CREATE UNLOGGED TABLE teams (
            code                    NUMERIC, -- permanent team code
            season                  NUMERIC,
            id                      NUMERIC, -- 23/24 season team code
            name                    TEXT,
            strength                NUMERIC,
            strength_overall_home   NUMERIC,
            strength_overall_away   NUMERIC,
            strength_attack_home    NUMERIC,
            strength_attack_away    NUMERIC,
            strength_defence_home   NUMERIC,
            strength_defence_away   NUMERIC
        );"""))
# creating our schema  and sending the table to psql
with engine.connect() as cursor:
    create_staging_table_teams(cursor)
teams.to_sql('teams', con=engine, if_exists='replace',index=False)

Index(['code', 'draw', 'form', 'id', 'loss', 'name', 'played', 'points',
       'position', 'short_name', 'strength', 'team_division', 'unavailable',
       'win', 'strength_overall_home', 'strength_overall_away',
       'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
       'strength_defence_away', 'pulse_id', 'season'],
      dtype='object')


20

## Fixtures Data

In [7]:

get=requests.get("https://fantasy.premierleague.com/api/fixtures/", headers=headers, timeout=5)
data=json.loads(get.text)

fixturedf = pd.DataFrame(data)
home_team = fixturedf[['code', 'event', 'finished', 'finished_provisional', 'id','kickoff_time', 'minutes', 'provisional_start_time', 'started','team_a', 'team_a_score', 'team_h', 'team_h_score','team_h_difficulty', 'team_a_difficulty']].copy()
home_team.rename(columns={'team_a':'opp_team', 'team_a_score':'opp_team_score', 'team_h':'team','team_h_score':'team_score','team_h_difficulty':'team_difficulty', 'team_a_difficulty':'opp_team_difficulty'}
                          ,inplace=True)
away_team = fixturedf[['code', 'event', 'finished', 'finished_provisional', 'id','kickoff_time', 'minutes', 'provisional_start_time', 'started', 'team_a', 'team_a_score', 'team_h', 'team_h_score','team_h_difficulty', 'team_a_difficulty']].copy()
away_team.rename(columns={'team_a':'team', 'team_a_score':'team_score', 'team_h':'opp_team','team_h_score':'opp_team_score','team_h_difficulty':'opp_team_difficulty', 'team_a_difficulty':'team_difficulty'}
                          ,inplace=True)
fixtures = pd.concat([home_team, away_team])
fixtures['season'] = season
print(fixtures.columns)
fixtures.head()

Index(['code', 'event', 'finished', 'finished_provisional', 'id',
       'kickoff_time', 'minutes', 'provisional_start_time', 'started',
       'opp_team', 'opp_team_score', 'team', 'team_score', 'team_difficulty',
       'opp_team_difficulty', 'season'],
      dtype='object')


,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,opp_team,opp_team_score,team,team_score,team_difficulty,opp_team_difficulty,season
0,2367552,NaN,False,False,15,None,0,False,None,6,NaN,12,NaN,2,2,20232024
1,2367713,NaN,False,False,176,None,0,False,None,4,NaN,13,NaN,3,5,20232024
2,2367538,1.0,True,True,1,2023-08-11T19:00:00Z,90,False,True,13,3.0,6,0.0,5,2,20232024
3,2367540,1.0,True,True,2,2023-08-12T12:00:00Z,90,False,True,16,1.0,1,2.0,2,4,20232024
4,2367539,1.0,True,True,3,2023-08-12T14:00:00Z,90,False,True,19,1.0,3,1.0,2,2,20232024


In [8]:
#Adding fixture information

def create_staging_table_fixtures(cursor):
    cursor.execute(text("""
        DROP TABLE IF EXISTS fixtures CASCADE;
        CREATE UNLOGGED TABLE fixtures (
            code                    NUMERIC,
            event                   NUMERIC,
            finished                BOOLEAN,
            finished_provisional    BOOLEAN,
            id                      NUMERIC,
            kickoff_time            TIMESTAMP,
            minutes                 NUMERIC,
            provisional_start_time  BOOLEAN,
            started                 BOOLEAN,
            opp_team                NUMERIC,
            opp_team_score          NUMERIC,
            team                    NUMERIC,
            team_score              NUMERIC,
            team_difficulty         NUMERIC,
            opp_team_difficulty     NUMERIC,
            season                  NUMERIC
        );"""))
# creating our schema  and sending the table to psql
with engine.connect() as cursor:
    create_staging_table_fixtures(cursor)

fixtures.to_sql('fixtures', con=engine, if_exists='replace',index=False)

760

## Player Details Data

In [9]:
#Player details dataframe

def get_player_details():
    #Get a list of columns for player details dataframe
    playerdetailurl="https://fantasy.premierleague.com/api/element-summary/1/"
    get=requests.get(playerdetailurl)
    p_detail_data=json.loads(get.text)      
    try:
        cols=list(p_detail_data['history'][0].keys())
    except IndexError:
        cols=None

    #Get data for each player, for each gameweek, for each column
    bootstrapurl="https://fantasy.premierleague.com/api/bootstrap-static/"
    get=requests.get(bootstrapurl)
    bootstrapdata=json.loads(get.text)

    #Get mp, max player
    mpl=[] #max player list
    for i in range(0,len(bootstrapdata['elements'])):
        mpl.append(bootstrapdata['elements'][i].get('id'))
    mp=max(mpl)

    e=[] #exceptions 1
    e1=[] #exceptions 2

    playerdetails=[]

    for playerid in range(1,mp+1):  
        try:
            print(playerid)
            playerdetailurl="https://fantasy.premierleague.com/api/element-summary/"+str(playerid)+"/"
            get=requests.get(playerdetailurl)
            p_detail_data=json.loads(get.text)

            for gw in range(0,39):    
                try:
                    row=p_detail_data['history'][gw]
                    r=[]
                    for col in cols:
                        r.append(row.get(col))
                    playerdetails.append(r)
                except IndexError as ex:
                    e.extend([playerid, gw,ex])    
        except Exception as ex1:
            e1.extend([playerid,gw,ex1])
    return playerdetails,cols
playerdetails,cols=get_player_details()

player_details_raw = pd.DataFrame(playerdetails, columns=cols)
player_details_raw['season'] = season
print(player_details_raw.columns)
player_details = player_details_raw[['season','element', 'fixture', 'opponent_team', 'total_points', 'was_home',
       'kickoff_time', 'team_h_score', 'team_a_score', 'round', 'minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out']]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
def create_staging_table_player_details(cursor):
    cursor.execute(text("""
        DROP TABLE IF EXISTS player_details CASCADE;
        CREATE UNLOGGED TABLE player_details (
            season            NUMERIC,
            element           NUMERIC,
            fixture           NUMERIC,
            opponent_team     NUMERIC,
            total_points      NUMERIC,
            was_home          BOOLEAN,
            kickoff_time      TIMESTAMP,
            team_h_score      NUMERIC,
            team_a_score      NUMERIC,
            round             NUMERIC,
            minutes           NUMERIC,
            goals_scored      NUMERIC,
            assists           NUMERIC,
            clean_sheets      NUMERIC,
            goals_conceded    NUMERIC,
            own_goals         NUMERIC,
            penalties_saved   NUMERIC,
            penalties_missed  NUMERIC,
            yellow_cards      NUMERIC,
            red_cards         NUMERIC,
            saves             NUMERIC,
            bonus             NUMERIC,
            bps               NUMERIC,
            influence         NUMERIC,
            creativity        NUMERIC,
            threat            NUMERIC,
            ict_index         NUMERIC,
            starts            NUMERIC,
            expected_goals    NUMERIC,
            expected_assists  NUMERIC,
            expected_goal_involvements NUMERIC,
            expected_goals_conceded NUMERIC,
            value             NUMERIC,
            transfers_balance NUMERIC,
            selected          NUMERIC,
            transfers_in      NUMERIC,
            transfers_out     NUMERIC
        );"""))
# creating our schema  and sending the table to psql
with engine.connect() as cursor:
    create_staging_table_player_details(cursor)
    
player_details.to_sql('player_details', con=engine, if_exists='replace',index=False)

172

## PlayerGW Data
* FPL API Summary player data changes each gameweek, so I need to request this data each week, append to the previous week's data and delete any duplicates

In [ ]:
# ## Create Table (Only needed once to commented out otherwise)

# # All FPL data (No fbref)
# # Join fixtures data to player data only for the current GW
# # Then add this data to previous GW's table

# create_table_query = """ 
#     CREATE TABLE playergw AS 
#         WITH playergw_cte AS (
#             WITH fixtures AS (
#                 SELECT
#                     code as fixture_code,
#                     event,
#                     finished,
#                     finished_provisional,
#                     id as fixture_id,
#                     kickoff_time,
#                     minutes as minutes_fixtures,
#                     provisional_start_time,
#                     started,
#                     opp_team as opp_team_id,
#                     opp_team_score,
#                     team as team_id,
#                     team_score,
#                     team_difficulty,
#                     opp_team_difficulty,
#                     season
#                 FROM fixtures
#             ),
#             players AS (
#                 SELECT 
#                     fbref_id,
#                     code,
#                     id,
#                     web_name,
#                     team as team_id_player,
#                     element_type,
#                     status,
#                     now_cost,
#                     transfers_in,
#                     transfers_in_event,
#                     transfers_out,
#                     transfers_out_event,
#                     event_points,
#                     goals_scored,
#                     assists,
#                     clean_sheets,
#                     goals_conceded,
#                     total_points,
#                     minutes,
#                     points_per_game,
#                     own_goals,
#                     penalties_saved,
#                     penalties_missed,
#                     yellow_cards,
#                     red_cards,
#                     saves,
#                     bonus,
#                     bps,
#                     influence,
#                     creativity,
#                     threat,
#                     ict_index,
#                     starts,
#                     form,
#                     ep_this,
#                     ep_next, 
#                     expected_goals,
#                     expected_assists,
#                     expected_goal_involvements,
#                     expected_goals_conceded,
#                     influence_rank,
#                     creativity_rank,
#                     threat_rank,
#                     ict_index_rank,
#                     corners_and_indirect_freekicks_order,
#                     direct_freekicks_order,
#                     penalties_order,
#                     expected_goals_per_90,
#                     saves_per_90,
#                     expected_assists_per_90,
#                     expected_goal_involvements_per_90,
#                     expected_goals_conceded_per_90,
#                     goals_conceded_per_90,
#                     points_per_game_rank,
#                     selected_rank,
#                     starts_per_90,
#                     clean_sheets_per_90,
#                     now_datetime_UTC,
#                     current_gw,
#                     next_gw
#                 FROM player
#             ),
#             opp_teams AS (
#                 SELECT 
#                     code as opp_team_code,
#                     id as opp_team_id,
#                     name as opp_team_name,
#                     strength as opp_strength,
#                     strength_overall_home as opp_strength_overall_home,
#                     strength_overall_away as opp_strength_overall_away,
#                     strength_attack_home as opp_strength_attack_home,
#                     strength_attack_away as opp_strength_attack_away,
#                     strength_defence_home as opp_strength_defence_home,
#                     strength_defence_away as opp_strength_defence_away
#                 FROM teams
#             ),
#             teams AS (
#                 SELECT
#                     code as team_code,
#                     id as team_id,
#                     name as team_name,
#                     strength,
#                     strength_overall_home,
#                     strength_overall_away,
#                     strength_attack_home,
#                     strength_attack_away,
#                     strength_defence_home,
#                     strength_defence_away
#                 FROM teams
#             )
#         SELECT *
#         FROM fixtures
#         LEFT JOIN teams
#             USING(team_id)
#         LEFT JOIN opp_teams
#             USING(opp_team_id)
#         INNER JOIN players
#             ON fixtures.team_id = players.team_id_player
#             AND fixtures.event = players.current_gw --fpl player summary data is grouped by event not by fixture
#         )
#         SELECT *
#         FROM playergw_cte;
#         """

# with engine.connect() as cursor:
#     result = cursor.execute(text(create_table_query))
#     if result:
#         print("Successfully created table!")
#         cursor.commit()

### Insert New Data

In [ ]:
# Add new data to previous GW's table

insert_table_query = """ 
    INSERT INTO playergw (
            opp_team_id,
            team_id,
            fixture_code,
            event,
            finished,
            finished_provisional,
            fixture_id,
            kickoff_time,
            minutes_fixtures,
            provisional_start_time,
            started,
            opp_team_score,
            team_score,
            team_difficulty,
            opp_team_difficulty,
            team_code,
            team_name,
            strength,
            strength_overall_home,
            strength_overall_away,
            strength_attack_home,
            strength_attack_away,
            strength_defence_home,
            strength_defence_away,
            opp_team_code,
            opp_team_name,
            opp_strength,
            opp_strength_overall_home,
            opp_strength_overall_away,
            opp_strength_attack_home,
            opp_strength_attack_away,
            opp_strength_defence_home,
            opp_strength_defence_away,
            fbref_id,
            code,
            id,
            web_name,
            team_id_player,
            element_type,
            status,
            now_cost,
            transfers_in,
            transfers_in_event,
            transfers_out,
            transfers_out_event,
            event_points,
            goals_scored,
            assists,
            clean_sheets,
            goals_conceded,
            total_points,
            minutes,
            points_per_game,
            own_goals,
            penalties_saved,
            penalties_missed,
            yellow_cards,
            red_cards,
            saves,
            bonus,
            bps,
            influence,
            creativity,
            threat,
            ict_index,
            starts,
            form,
            ep_this,
            ep_next,
            expected_goals,
            expected_assists,
            expected_goal_involvements,
            expected_goals_conceded,
            influence_rank,
            creativity_rank,
            threat_rank,
            ict_index_rank,
            corners_and_indirect_freekicks_order,
            direct_freekicks_order,
            penalties_order,
            expected_goals_per_90,
            saves_per_90,
            expected_assists_per_90,
            expected_goal_involvements_per_90,
            expected_goals_conceded_per_90,
            goals_conceded_per_90,
            points_per_game_rank,
            selected_rank,
            starts_per_90,
            clean_sheets_per_90,
            now_datetime_utc,
            current_gw,
            next_gw
    )
        WITH playergw_cte AS (
            WITH fixtures AS (
                SELECT
                    code as fixture_code,
                    event,
                    finished,
                    finished_provisional,
                    id as fixture_id,
                    kickoff_time,
                    minutes as minutes_fixtures,
                    provisional_start_time,
                    started,
                    opp_team as opp_team_id,
                    opp_team_score,
                    team as team_id,
                    team_score,
                    team_difficulty,
                    opp_team_difficulty
                FROM fixtures
            ),
            players AS (
                SELECT 
                    fbref_id,
                    code,
                    id,
                    web_name,
                    team as team_id_player,
                    element_type,
                    status,
                    now_cost,
                    transfers_in,
                    transfers_in_event,
                    transfers_out,
                    transfers_out_event,
                    event_points,
                    goals_scored,
                    assists,
                    clean_sheets,
                    goals_conceded,
                    total_points,
                    minutes,
                    points_per_game,
                    own_goals,
                    penalties_saved,
                    penalties_missed,
                    yellow_cards,
                    red_cards,
                    saves,
                    bonus,
                    bps,
                    influence,
                    creativity,
                    threat,
                    ict_index,
                    starts,
                    form,
                    ep_this,
                    ep_next,
                    expected_goals,
                    expected_assists,
                    expected_goal_involvements,
                    expected_goals_conceded,
                    influence_rank,
                    creativity_rank,
                    threat_rank,
                    ict_index_rank,
                    corners_and_indirect_freekicks_order,
                    direct_freekicks_order,
                    penalties_order,
                    expected_goals_per_90,
                    saves_per_90,
                    expected_assists_per_90,
                    expected_goal_involvements_per_90,
                    expected_goals_conceded_per_90,
                    goals_conceded_per_90,
                    points_per_game_rank,
                    selected_rank,
                    starts_per_90,
                    clean_sheets_per_90,
                    now_datetime_UTC,
                    current_gw,
                    next_gw
                FROM player
            ),
            opp_teams AS (
                SELECT 
                    code as opp_team_code,
                    id as opp_team_id,
                    name as opp_team_name,
                    strength as opp_strength,
                    strength_overall_home as opp_strength_overall_home,
                    strength_overall_away as opp_strength_overall_away,
                    strength_attack_home as opp_strength_attack_home,
                    strength_attack_away as opp_strength_attack_away,
                    strength_defence_home as opp_strength_defence_home,
                    strength_defence_away as opp_strength_defence_away
                FROM teams
            ),
            teams AS (
                SELECT
                    code as team_code,
                    id as team_id,
                    name as team_name,
                    strength,
                    strength_overall_home,
                    strength_overall_away,
                    strength_attack_home,
                    strength_attack_away,
                    strength_defence_home,
                    strength_defence_away
                FROM teams
            )
        SELECT *
        FROM fixtures
        LEFT JOIN teams
            USING(team_id)
        LEFT JOIN opp_teams
            USING(opp_team_id)
        INNER JOIN players
            ON fixtures.team_id = players.team_id_player
            AND fixtures.event = players.current_gw --fpl player summary data is grouped by event not by fixture
        )
        SELECT *
        FROM playergw_cte;
        """

with engine.connect() as cursor:
    result = cursor.execute(text(insert_table_query))
    if result:
        print("Successfully added rows")
        cursor.commit()

Successfully added rows


In [ ]:
#Delete Duplicates
delete_duplicates_query = """
        WITH duplicates AS (
            SELECT *,
                ROW_NUMBER() OVER(
                    PARTITION BY
                        code,
                        event,
                        fixture_id,
                        now_datetime_utc
                    ORDER BY 
                        now_datetime_utc) as rn
            FROM playergw)

        DELETE FROM playergw 
        WHERE (code, event, fixture_id, now_datetime_utc) IN (
            SELECT code, event, fixture_id, now_datetime_utc
            FROM duplicates
            WHERE rn <> 1)
        ;
        """
with engine.connect() as cursor:
    result = cursor.execute(text(delete_duplicates_query))
    if result:
        print("Successfully deleted duplicates")
        cursor.commit()

Successfully deleted duplicates


## Join Data Together & Save to File

In [ ]:
#Join non-duplicates with player_details data, only keep 1 reord per player per event per fixture
query = """
        WITH duplicates AS (
            SELECT *,
            ROW_NUMBER() OVER(
                    PARTITION BY
                        code,
                        event,
                        fixture_id
                    ORDER BY 
                        now_datetime_utc DESC) as rn --Keep the most recent
            FROM playergw)
        SELECT *
        FROM duplicates
        LEFT JOIN player_details
        ON duplicates.id = player_details.element
            AND duplicates.event = player_details.round
        WHERE rn = 1
        ;
        """
with engine.connect() as conn:
    playergw = pd.read_sql_query(query, conn)

playergw.to_csv("data/playergw_updated", index=False)
playergw.to_csv(f"data/playergw_{version}", index=False)
print(playergw.shape)
playergw.head()

(1952, 132)


,opp_team_id,team_id,fixture_code,event,finished,finished_provisional,fixture_id,kickoff_time,minutes_fixtures,provisional_start_time,...,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,10,9,2367542,1.0,True,True,5,2023-08-12T14:00:00Z,90,False,...,0.0,0.00,0.00,0.00,0.00,40.0,0.0,5342.0,0.0,0.0
1,2,9,2367548,2.0,True,True,11,2023-08-20T13:00:00Z,90,False,...,0.0,0.00,0.00,0.00,0.00,40.0,-150.0,6537.0,179.0,329.0
2,20,9,2367564,3.0,True,True,27,2023-08-26T14:00:00Z,90,False,...,0.0,0.00,0.00,0.00,0.00,40.0,-250.0,6570.0,95.0,345.0
3,12,5,2367541,1.0,True,True,4,2023-08-12T14:00:00Z,90,False,...,1.0,0.00,0.02,0.02,0.41,50.0,0.0,17570.0,0.0,0.0
4,20,5,2367557,2.0,True,True,20,2023-08-19T14:00:00Z,90,False,...,1.0,0.00,0.13,0.13,0.91,50.0,482.0,23898.0,2661.0,2179.0


In [ ]:
#Show top-scoring player and every single scrape in the database
query = """
        WITH playergw AS (
            SELECT web_name,id, event, event_points, strength_overall_home,ep_next, now_datetime_utc, 
            ROW_NUMBER() OVER(
                    PARTITION BY
                        code,
                        event,
                        fixture_id,
                        current_gw
                        --EXTRACT(DAY FROM now_datetime_utc) --Only one record per day
                    ORDER BY 
                        now_datetime_utc DESC) as rn --Only select the latest date
            FROM playergw
        ),
        best_players AS (
            SELECT
                id
            FROM playergw
            WHERE rn=1
            GROUP BY id
            ORDER BY sum(event_points) DESC
            LIMIT 1
        )
        SELECT * 
        FROM playergw
        WHERE id IN (SELECT id FROM best_players)
        ORDER BY id, event, now_datetime_utc         
        ;
        """
with engine.connect() as conn:
    df = pd.read_sql_query(query, conn)
df.tail(20)

,web_name,id,event,event_points,strength_overall_home,ep_next,now_datetime_utc,rn
0,March,140,1.0,9,1165,2.3,2023-08-18 13:49:23.059834+00:00,1
1,March,140,2.0,0,1165,4.5,2023-08-18 18:39:14.427682+00:00,9
2,March,140,2.0,0,1165,4.5,2023-08-18 20:03:02.974600+00:00,8
3,March,140,2.0,1,1165,5.0,2023-08-19 14:20:23.884260+00:00,7
4,March,140,2.0,12,1165,10.5,2023-08-19 16:19:11.936856+00:00,6
5,March,140,2.0,12,1165,10.5,2023-08-19 16:52:26.714179+00:00,5
6,March,140,2.0,15,1165,12.0,2023-08-20 20:02:08.670008+00:00,4
7,March,140,2.0,15,1165,12.0,2023-08-22 11:17:05.406761+00:00,3
8,March,140,2.0,15,1165,12.0,2023-08-22 20:42:34.842482+00:00,2
9,March,140,2.0,15,1165,12.0,2023-08-23 23:09:53.467495+00:00,1
